# Interactive Plots of COVID-19 Data
This is a notebook to interact with COVID-19 data using [Jupyter](https://jupyter.org/) and [Hvplot](https://hvplot.holoviz.org/). Currently we are focused on data from the US but may expand our analyses in the near future.

## Load Johns Hopkins COVID-19 Data
Here we load the COVID-19 confirmed case data from the [The Center for Systems Science and Engineering (CSSE)](https://systems.jhu.edu) at Johns Hopkins University. The CCSE COVID-19 [GitHub Repo](https://github.com/CSSEGISandData/COVID-19) has more information about these data and their sources.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1000)
import hvplot.pandas

In [ ]:
src = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

In [ ]:
df = pd.read_csv(src)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

In [ ]:
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None

In [ ]:
df.state = state
df.insert(0, 'county', county)
df.head()

## Plot All US Cases on Log Scale
Below is a quick plot of all confirmed cases in the US on a logarithmic scale. 

Hvplot creates holoviews objects, and the `*` symbol means [overlay](http://holoviews.org/reference/containers/bokeh/Overlay.html).  See [holoviz plot customization](http://holoviews.org/user_guide/Customizing_Plots.html) for available options.  

In [ ]:
opts = {'legend': False, 'logy': True, 'grid': True, 'width': 700, 'height': 400,
        'title': 'Confirmed cases of COVID-19 in the US', 'padding':0.1 }
lines = df.select_dtypes(np.int).sum().hvplot(**opts) 
dots  = df.select_dtypes(np.int).sum().hvplot.scatter(**opts)
usa = lines * dots
usa

## Single State Example
Here is an example of plotting data from a single US state.

In [ ]:
MA = df[(df.state=='MA') | (df.state=='Massachusetts')]
opts = {'legend': False, 'logy': True, 'grid': True, 'width': 700, 'height': 400,
        'title': 'Confirmed cases of COVID-19 in Massachusetts', 'padding':0.1,
        'ylim':(1.0,1.0e3) }
lines = MA.select_dtypes(np.int).sum().hvplot(**opts)
dots = MA.select_dtypes(np.int).sum().hvplot.scatter(**opts)
lines * dots

## Multiple Region Example

Turn the above code for a single state into a function so it's easier to explore several states

In [ ]:
def state(name='Massachusetts', code='MA'):
    state = df[(df.state==name) | (df.state==code)]
    opts = {'legend': False, 'logy': True, 'grid': True, 'width': 700, 'height': 400,
        'title': f'Confirmed cases of COVID-19 in {code}', 'padding':0.1,
        'ylim':(1.0,1.0e3) }
    lines = state.select_dtypes(np.int).sum().hvplot(**opts)
    dots = state.select_dtypes(np.int).sum().hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

In [ ]:
ma = state(name='Massachusetts', code='MA')

In [ ]:
mi = state(name='Michigan', code='MI')

In [ ]:
pa = state(name='Pennsylvania', code='PA')

In [ ]:
(ma * mi * pa * usa).opts(title_format='Confirmed cases of COVID-19', ylim=(1.0,1.0e4))